In [1]:
# Can be autimated across quarter if needed
# To be noted that rolled up to store levels first, and then sum up which results in inflation 
import pandas as pd
import numpy as np
import datetime
import os
import glob


def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)

In [2]:
# Fix fisrt week
last_saturday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday()+2)


First_week_ending_Q3_2019=datetime.date(2019,8,10)

last_year_week=last_saturday-datetime.timedelta(days=52*7)

Nth_week=int((last_saturday-First_week_ending_Q3_2019).days/7)+1

write_folder="/home/jian/Projects/Big_Lots/Analysis/2019_Q3/BL_Excutive_Dashboard/output/output_"+str(last_saturday)+"/"
try:
    os.stat(write_folder)
except:
    os.mkdir(write_folder)
str(last_saturday)
Nth_week

11

In [3]:
# Checking for the recent weeks that haven't moved to the folder
possible_recent_folders="/home/jian/BigLots/MediaStorm_"+str(last_saturday)+"/"
daily_data_recent=[x for x in list(recursive_file_gen(possible_recent_folders)) if ("aily" in x) & (".txt" in x)]

archived_folders_2018="/home/jian/BigLots/2018_by_weeks/MediaStorm_"+str(last_saturday)+"/"
daily_data_2018_achived=[x for x in list(recursive_file_gen(archived_folders_2018)) if ("aily" in x) & (".txt" in x)]

archived_folders_2019="/home/jian/BigLots/2019_by_weeks/MediaStorm_"+str(last_saturday)+"/"
daily_data_2019_achived=[x for x in list(recursive_file_gen(archived_folders_2019)) if ("aily" in x) & (".txt" in x)]

daily_data_list=daily_data_recent+daily_data_2018_achived+daily_data_2019_achived
if len(daily_data_list)!=1:
    print("Error of Daily Sales Data Path")
else:
    daily_data_path_this_year=daily_data_list[0]

In [4]:
daily_data_list_hist=list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/"))
daily_data_list_hist=[x for x in daily_data_list_hist if (str(last_year_week) in x)&("DailySales" in x)]
if len(daily_data_list_hist)!=1:
    print("Error of Daily Sales Data Path")
else:
    daily_data_path_last_year=daily_data_list_hist[0]

In [5]:
# Actually it's 2019Q1 as in the excel
Df_Quadrant=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2019_Q3/BL_Excutive_Dashboard/Excel_BL_2019_Q2_post_quadrants_JL_2019-08-13.xlsx",
                                 dype=str,sheetname="2019_Q1_Store_Quad_Defination",usecols=['location_id','Quadrant'])
Df_Quadrant['location_id']=Df_Quadrant['location_id'].astype(str)
Df_Quadrant.shape

(1355, 2)

In [6]:
def agg_daily_data_by_store_subclass_level(file_path_daily):
    df=pd.read_table(file_path_daily,dtype=str,sep="|",usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','subclass_transaction_amt'])
    df=df[df['location_id']!="6990"] # 6990 removed
    df['subclass_transaction_amt']=df['subclass_transaction_amt'].astype(float)
    df['Reward_Level']=np.where(pd.isnull(df['customer_id_hashed']),"Non_Rewards","Rewards")
    
    print("Total_Sales:",df['subclass_transaction_amt'].sum())
    df_sales=df.groupby(['location_id','Reward_Level'])['subclass_transaction_amt'].sum().to_frame().reset_index().rename(columns={"subclass_transaction_amt":"Sales"})
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','Reward_Level']].drop_duplicates().groupby(['location_id','Reward_Level'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"Transactions"})
    
    df=df[df['Reward_Level']=="Rewards"]
    df_ids=df.groupby(['location_id','Reward_Level'])['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"shopped_unique_ids_in_the_week"})
    
    df=pd.merge(df_sales,df_trans,on=["location_id",'Reward_Level'],how="outer")
    df=pd.merge(df,df_ids,on=["location_id",'Reward_Level'],how="outer")
    
    return df

In [7]:
def agg_daily_data_by_store_item_level(file_path_daily):
    df=pd.read_table(file_path_daily,dtype=str,sep="|",usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','item_transaction_amt'])
    df=df[df['location_id']!="6990"] # 6990 removed
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['Reward_Level']=np.where(pd.isnull(df['customer_id_hashed']),"Non_Rewards","Rewards")
    
    print("Total_Sales:",df['item_transaction_amt'].sum())
    df_sales=df.groupby(['location_id','Reward_Level'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"Sales"})
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','Reward_Level']].drop_duplicates().groupby(['location_id','Reward_Level'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"Transactions"})
    
    df=df[df['Reward_Level']=="Rewards"]
    df_ids=df.groupby(['location_id','Reward_Level'])['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"shopped_unique_ids_in_the_week"})
    
    df=pd.merge(df_sales,df_trans,on=["location_id",'Reward_Level'],how="outer")
    df=pd.merge(df,df_ids,on=["location_id",'Reward_Level'],how="outer")
    
    return df

In [8]:
daily_data_path_last_year

'/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-10-20/MediaStormDailySales20181023-113036-640.txt'

In [9]:
df_daily_this_year_by_store=agg_daily_data_by_store_item_level(daily_data_path_this_year)
df_daily_last_year_by_store=agg_daily_data_by_store_subclass_level(daily_data_path_last_year)



Total_Sales: 103388350.83
Total_Sales: 81263299.9


In [10]:
df_daily_this_year_by_store.columns=df_daily_this_year_by_store.columns.tolist()[:2]+["This_Year_"+x for x in df_daily_this_year_by_store.columns.tolist()[2:]]
df_daily_last_year_by_store.columns=df_daily_last_year_by_store.columns.tolist()[:2]+["Last_Year_"+x for x in df_daily_last_year_by_store.columns.tolist()[2:]]
both_year=pd.merge(df_daily_this_year_by_store,df_daily_last_year_by_store,on=['location_id','Reward_Level'],how="outer")
both_year=pd.merge(both_year,Df_Quadrant,on="location_id",how="left")



In [11]:
both_year_exlusion_last_year=both_year[(pd.isnull(both_year['Last_Year_Sales'])) | (pd.isnull(both_year['Last_Year_Transactions']))]
both_year_exlusion_this_year=both_year[(pd.isnull(both_year['This_Year_Sales'])) | (pd.isnull(both_year['This_Year_Transactions']))]
exlusion_no_quad=both_year[pd.isnull(both_year['Quadrant'])]

all_exclusion_stores=set(both_year_exlusion_last_year['location_id'].tolist()+both_year_exlusion_this_year['location_id'].tolist()+exlusion_no_quad['location_id'].tolist())


In [12]:
both_year_inclusions=both_year[~both_year['location_id'].isin(all_exclusion_stores)]

both_year_inclusions_Total=both_year_inclusions.groupby(['Reward_Level'])['This_Year_Sales','This_Year_Transactions','This_Year_shopped_unique_ids_in_the_week','Last_Year_Sales','Last_Year_Transactions','Last_Year_shopped_unique_ids_in_the_week'].sum().reset_index()
both_year_inclusions_Total["Summary_Level"]="Total"
both_year_inclusions_Total_store_counts=both_year_inclusions.groupby(['Reward_Level'])['location_id'].nunique().reset_index().rename(columns={"location_id":"store_counts"})
both_year_inclusions_Total=pd.merge(both_year_inclusions_Total,both_year_inclusions_Total_store_counts,on="Reward_Level")

both_year_inclusions_Quad=both_year_inclusions.groupby(['Reward_Level','Quadrant'])['This_Year_Sales','This_Year_Transactions','This_Year_shopped_unique_ids_in_the_week','Last_Year_Sales','Last_Year_Transactions','Last_Year_shopped_unique_ids_in_the_week'].sum().reset_index()
both_year_inclusions_Quad_store_counts=both_year_inclusions.groupby(['Reward_Level','Quadrant'])['location_id'].nunique().reset_index().rename(columns={"location_id":"store_counts"})
both_year_inclusions_Quad=pd.merge(both_year_inclusions_Quad,both_year_inclusions_Quad_store_counts,on=["Reward_Level","Quadrant"])
both_year_inclusions_Quad=both_year_inclusions_Quad.rename(columns={"Quadrant":"Summary_Level"})

output=both_year_inclusions_Total.append(both_year_inclusions_Quad)
output=output.sort_values(['Summary_Level','Reward_Level'],ascending=[True,False])

output=output[output['Summary_Level']=="Total"].append(output[output['Summary_Level']!="Total"])

output_sales_both_R_N=output.groupby(['Summary_Level'])['This_Year_Sales'].sum().to_frame().reset_index().rename(columns={"This_Year_Sales":"This_week_Total_R_and_N"})
output=pd.merge(output,output_sales_both_R_N,on="Summary_Level")



In [13]:
output['YoY_Incr_Sales']=np.round((output['This_Year_Sales']-output['Last_Year_Sales'])/output['Last_Year_Sales'],4)
output['YoY_Incr_Rew_Shoppers']=np.round((output['This_Year_shopped_unique_ids_in_the_week']-output['Last_Year_shopped_unique_ids_in_the_week'])/output['Last_Year_shopped_unique_ids_in_the_week'],4)
output['YoY_Incr_Transactions']=np.round((output['This_Year_Transactions']-output['Last_Year_Transactions'])/output['Last_Year_Transactions'],4)
output['Avg_Order_Value']=np.round(output['This_Year_Sales']/output['This_Year_Transactions'],4)

output['Rew/Non-Rew_Share_of_Sales']=np.round(output['This_Year_Sales']/output['This_week_Total_R_and_N'],4)
output['Week_Ending_Date']=str(last_saturday)

In [14]:
output_final=output[['Week_Ending_Date','Summary_Level','Reward_Level','store_counts','YoY_Incr_Sales','YoY_Incr_Rew_Shoppers','YoY_Incr_Transactions','Avg_Order_Value',
                     'Rew/Non-Rew_Share_of_Sales','This_Year_Transactions','This_Year_Sales','This_Year_shopped_unique_ids_in_the_week']].rename(columns={"This_Year_Transactions":"Transactions_last_7_days",
                     "This_Year_Sales":"Sales_last_7_days","This_Year_shopped_unique_ids_in_the_week":"Shopped_Rew_Unique_IDs_last_7_days"})


In [15]:
exclusion_1=exlusion_no_quad[['location_id']]
exclusion_1['exclusion']="Not_Defined_Quadrant"

exclusion_2=both_year_exlusion_last_year[['location_id']]
exclusion_2['exclusion']="Lack_of_week_last_year"

exclusion_3=both_year_exlusion_this_year[['location_id']]
exclusion_3['exclusion']="Lack_of_week_this_year"

exclusion_df=exclusion_3.append(exclusion_2).append(exclusion_1).drop_duplicates('location_id')
exclusion_df['Week_Ending_Date']=str(last_saturday)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

In [16]:
def int_ceil_not_NA(x):
    if ~np.isnan(x):
        y=int(np.ceil(x))
    else:
        y=x
    return y


output_final=output_final.rename(columns={"Week_Ending_Date":"Week Ending Date","Summary_Level":"Summary Level","Reward_Level":"Reward Level",
                         "store_counts":"# Stores","YoY_Incr_Sales":"YOY incr Sales/Store","YoY_Incr_Rew_Shoppers":"YOY incr Rew Shoppers/Store",
                         "YoY_Incr_Transactions":"YOY incr Transactions/Store","Avg_Order_Value":"Avg Order Value",
                         "Rew/Non-Rew_Share_of_Sales":"Rew share of Sales","Transactions_last_7_days":"Transactions last 7 days",
                         "Sales_last_7_days":"Total Sales Last 7 Days","Shopped_Rew_Unique_IDs_last_7_days":"Shopped Rew IDs Last 7 days"})
output_final['Reward Level']=output_final['Reward Level'].replace("Non_Rewards","Non-Rewards")
output_final['Avg # of Reward IDs shopped per store']=output_final['Shopped Rew IDs Last 7 days']/output_final['# Stores']

output_final['Avg # of Reward IDs shopped per store']=output_final['Avg # of Reward IDs shopped per store'].apply(lambda x: int_ceil_not_NA(x))
output_final['Weekly Cost']=np.nan

In [17]:
output_final.to_csv(write_folder+"output_"+str(last_saturday)+".csv",index=False)
exclusion_df.to_csv(write_folder+"exclusion_stores_"+str(last_saturday)+".csv",index=False)
both_year_inclusions.to_csv(write_folder+"inclusion_by_store_"+str(last_saturday)+".csv",index=False)

In [18]:
Simeng_recent_weekly_data_folder="/home/simeng/outputs_"+str(last_saturday)+"/"

output_final.to_csv(Simeng_recent_weekly_data_folder + 'output_quadrant.csv',index = False)

In [19]:
print(datetime.datetime.now())

2019-10-22 17:08:06.675020


In [20]:
output_final

,Week Ending Date,Summary Level,Reward Level,# Stores,YOY incr Sales/Store,YOY incr Rew Shoppers/Store,YOY incr Transactions/Store,Avg Order Value,Rew share of Sales,Transactions last 7 days,Total Sales Last 7 Days,Shopped Rew IDs Last 7 days,Avg # of Reward IDs shopped per store,Weekly Cost
0,2019-10-19,Total,Rewards,1329,0.5380,0.3130,0.3757,40.0678,0.6904,1654641.0,66297874.89,1341272.0,1010.0,NaN
1,2019-10-19,Total,Non-Rewards,1329,-0.1126,NaN,-0.1856,25.8918,0.3096,1148244.0,29730067.80,NaN,NaN,NaN
2,2019-10-19,Quadrant I,Rewards,379,0.5873,0.3585,0.4286,40.3525,0.6992,510156.0,20586078.72,410812.0,1084.0,NaN
3,2019-10-19,Quadrant I,Non-Rewards,379,-0.1358,NaN,-0.2029,25.4604,0.3008,347900.0,8857673.23,NaN,NaN,NaN
4,2019-10-19,Quadrant II,Rewards,161,0.5058,0.3196,0.3785,35.8858,0.6794,224549.0,8058128.05,181796.0,1130.0,NaN
5,2019-10-19,Quadrant II,Non-Rewards,161,-0.1700,NaN,-0.2138,23.5253,0.3206,161614.0,3802023.53,NaN,NaN,NaN
6,2019-10-19,Quadrant III,Rewards,411,0.4885,0.2829,0.3450,39.8813,0.6802,472390.0,18839526.35,384608.0,936.0,NaN
7,2019-10-19,Quadrant III,Non-Rewards,411,-0.1075,NaN,-0.1788,26.2320,0.3198,337666.0,8857661.22,NaN,NaN,NaN
8,2019-10-19,Quadrant IV,Rewards,378,0.5510,0.2930,0.3499,42.0385,0.6961,447546.0,18814141.77,364056.0,964.0,NaN
9,2019-10-19,Quadrant IV,Non-Rewards,378,-0.0612,NaN,-0.1560,27.2790,0.3039,301064.0,8212709.82,NaN,NaN,NaN
